In [1]:
%load_ext line_profiler

import importlib
from tqdm import tqdm

import numpy as np
from numpy.random import RandomState
from numpy.testing import assert_allclose, assert_array_equal
from scipy import sparse
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix
from scipy.stats import norm

from matplotlib import pyplot as plt
%matplotlib tk

import network as N

rng_seed = 1234
rng = np.random.RandomState(rng_seed)

In [2]:
# new, useful code!

from itertools import combinations

def assert_allcombs_equal(iter_arr, ck_fun=assert_allclose):
    combs = combinations(iter_arr, 2)
    for comb in combs:
        ck_fun(*comb)

def imshow_cb(a, ax):
    
    i = ax.imshow(a, cmap='RdBu_r')
    cb = plt.colorbar(i)

    lims = cb.get_clim()
    maxabs = np.fabs(lims).max()
    cb.set_clim(-maxabs, maxabs)

In [4]:
importlib.reload(N)

<module 'network' from '/export/home/mike/python/py-rrn/network.py'>

In [3]:
GENERATOR_PARAMS = {'n_units': 800,
                 'p_plastic': 0.6,
                 'p_connect': 0.1,
                 'syn_strength': 1.5,
                 'tau_ms': 10,
                 'sigmoid': np.tanh,
                 'noise_amp': 0.001}

TRIAL_PARAMS = {'length_ms': 1000,
                'spacing': 2,
                'time_step': 1,
                'start_train_ms': 250,
                'end_train_ms': 1400,}

INPUT_PARAMS = {'n_units': 1,
                'value': 5,
                'start_ms': 200,
                'duration_ms': 50}

OUTPUT_PARAMS = {'n_units': 1,
                 'value': 1,
                'center_ms': 1250,
                'width_ms': 30,
                'baseline_val': 0.2}

TRAIN_PARAMS = {'n_trials_recurrent': 20,
                'n_trials_readout': 10,
                'n_trials_test': 10}

GEN = N.Network(**GENERATOR_PARAMS)
TRYAL = N.Trial(**TRIAL_PARAMS)
IN = N.Input(TRYAL, **INPUT_PARAMS)
OUT = N.Output(TRYAL, **OUTPUT_PARAMS)
TRAIN = N.Trainer(GEN, In, Out, TRYAL, **TRAIN_PARAMS)

In [4]:
GEN.initialize_weights()

In [6]:
## connectivity matrices

# "generator" network recurrent weight matrix (WXX)
# indices are define as WXX[postsyn, presyn]

# logical mask for non-zero connections
WXX_mask = np.random.rand(GEN.n_units, GEN.n_units)  # uniform distribution!
WXX_mask[WXX_mask <= GEN.p_connect] = 1
WXX_mask[WXX_mask < 1] = 0

# connection weights
WXX_vals = np.random.normal(scale=GEN.scale_recurr, size=(GEN.n_units, GEN.n_units))

# create non-sparse version of WXX and set self-connections (diagonal elements) to 0
WXX_nonsparse = WXX_vals * WXX_mask
np.fill_diagonal(WXX_nonsparse, 0)

# convert to be sparse
WXX = csr_matrix(WXX_nonsparse)
WXX_c = csc_matrix(WXX_nonsparse) # for testing
WXX_o = coo_matrix(WXX_nonsparse) # for testing

# make a copy
WXX_ini = WXX.copy()

# input => generator weights
WInputX = np.random.normal(scale=1, size=(GEN.n_units, IN.n_units))

# generator weights => output
WXOut = np.random.normal(scale=1/np.sqrt(GEN.n_units), size=(OUT.n_units, GEN.n_units))

# make a copy
WXOut_ini = WXOut.copy()

In [7]:
# check type size and range of these mats

# ck_mats = (WXX_nonsparse, WXX, WXX_c, WXX_o, WInputX, WXOut)
ck_mats = (WXX, WInputX, WXOut)

for cm in ck_mats:
    print(type(cm), cm.shape, np.min(cm), np.max(cm))

<class 'scipy.sparse.csr.csr_matrix'> (800, 800) -0.74757972326 0.719404408995
<class 'numpy.ndarray'> (800, 1) -3.11938308122 2.99942195034
<class 'numpy.ndarray'> (1, 800) -0.101989618004 0.127104372032


In [8]:
# trial training time indices

print(TRYAL.start_train_n)
print(TRYAL.end_train_n)

250
1400


In [9]:
# tracking histories

# vectors representing the activity of the RRN units and ouputs over time
X_history = np.zeros((GEN.n_units, TRYAL.n_steps))
Out_history = np.zeros((OUT.n_units, TRYAL.n_steps))

# logging changes and such
WXOut_len = np.zeros((TRYAL.n_steps))
WXX_len = np.zeros((TRYAL.n_steps))
dW_readout_len = np.zeros((TRYAL.n_steps))
dW_recurr_len = np.zeros((TRYAL.n_steps))
train_window = 0

In [10]:
# check type size and range of these mats

ck_mats = (X_history, Out_history, WXOut_len, WXX_len, dW_readout_len, dW_recurr_len)

for cm in ck_mats:
    print(type(cm), cm.shape, np.min(cm), np.max(cm))

<class 'numpy.ndarray'> (800, 1600) 0.0 0.0
<class 'numpy.ndarray'> (1, 1600) 0.0 0.0
<class 'numpy.ndarray'> (1600,) 0.0 0.0
<class 'numpy.ndarray'> (1600,) 0.0 0.0
<class 'numpy.ndarray'> (1600,) 0.0 0.0
<class 'numpy.ndarray'> (1600,) 0.0 0.0


In [11]:
# initial conditions

# initial Xv is random uniform distribution from -1 to +1
# this represents an analog firing rate
Xv = 2 * np.random.rand(GEN.n_units, 1) - 1

# X is the sigmoid (tanh) of Xv, which will be bound from -0.76 to +0.76
# which represents a membrane potential
# as firing rate increases,
# membrane potential increases less quickly than linearly
X = GEN.sigmoid(Xv)

# O represents the output, where each output is the output-weighted membrane potential of each neuron
# O is random normal from -0.1 to 0.1
O = np.zeros((OUT.n_units,1))

In [12]:
# check type size and range of these mats

ck_mats = (Xv, X, O)

for cm in ck_mats:
    print(type(cm), cm.shape, np.min(cm), np.max(cm))

<class 'numpy.ndarray'> (800, 1) -0.999089812107 0.994207731077
<class 'numpy.ndarray'> (800, 1) -0.761211635339 0.759150800437
<class 'numpy.ndarray'> (1, 1) 0.0 0.0


In [13]:
# what does the sigmoid do?

s = np.linspace(-1, 1, 100)
h = GEN.sigmoid(s)

f, ax = plt.subplots()

ax.plot(s, h)
# ax.plot(s, s, 'k--')
ax.set_xlabel('Firing Rate')
ax.set_ylabel('Membrane Potential (uV)')

ax.legend()

/usr/local/anaconda3/lib/python3.5/site-packages/matplotlib/axes/_axes.py:531: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [14]:
TRAIN_RECURR = False
TRAIN_READOUT = False

In [77]:
# integration loop

# constant value by which the update to Xv based on the summation
# of recurrent generator network inputs AND external input inputs
# AND noise, are divided...
# this simulates a neural time constant?
use_noiseamp = 0
time_div = GEN.tau_ms / TRYAL.time_step

for i in tqdm(range(TRYAL.n_steps)):

    # update units
    
    # (IN.n_units, 1)
    in_vec = IN.series[:, i]
    
    # (GEN.n_units, 1)
    noise = use_noiseamp * np.random.normal(scale=np.sqrt(TRYAL.time_step), size=(GEN.n_units,1))
    
    
#     Xv_current = \
#         WXX * X \ # (GEN.n_units, GEN.n_units) * (GEN.n_units, 1) => (GEN.n_units, 1)
#         + \
#         WInputX * in_vec \ # (GEN.n_units, IN.n_units) * (IN.n_units, 1) => (GEN.n_units, 1)
#         + \
#         noise # (GEN.n_units, 1)

    # note that X is the "previous" membrane potential (random sigmoid vector scaled to .76)
    # WXX * X is the weighted previous membrane potential (sparse random normal square matrix scaled to ~0.65 times X)
    # WInputX * in_vec is the weighted input (random normal vector scaled to 4 times current input vector/scalar)
    Xv_current = WXX * X + WInputX * in_vec + noise
    
#     Xv += \  # (GEN.n_units, 1)
#         (-Xv + Xv_current) \ (GEN.n_units, 1) + (GEN.n_units, 1) => (GEN.n_units, 1)
#         / \
#         time_div (scalar)
    # Xv is previous firing rate
    # we take the negative difference between that firing rate and the summed incoming membrane potentials,
    # and divide by the time constant of the network (in this case, 10 steps)
    # then add that the current firing rate
    # the idea being that the change to the current firing rate in each step
    # is the summed input of weighted membrane potentials from presynaptic units
    Xv += (-Xv + Xv_current) / time_div
    
    # then we convert that newly updated firing rate back into a sigmoid
    # to determine the new membrane potential of all neurons
    X = GEN.sigmoid(Xv) # (GEN.n_units, 1) => (GEN.n_units, 1)
    
    # then we determine all of those neurons outputs as the dot-product of their
    # membrane potentials and output weights
    O = np.dot(WXOut, X) # (Out.n_units, GEN.n_units) *dot* (GEN.n_units, 1) => (Out.n_units, 1)

    # start-end training window
    if (i == TRYAL.start_train_n):
        train_window = True
    if (i == TRYAL.end_train_n):
        train_window = False

    # training
    if train_window and i % TRYAL.spacing == 0:

        if TRAIN_RECURR:
            # train recurrent
            error = X - Target_innate_X[:, i]
            for plas in 1:GEN.n_plastic
#                 X_pre_plastic = X(pre_plastic_units(plas).inds)
#                 P_recurr_old = P_recurr(plas).P
#                 P_recurr_old_X = P_recurr_old*X_pre_plastic
#                 den_recurr = 1 + X_pre_plastic'*P_recurr_old_X
#                 P_recurr(plas).P = P_recurr_old - (P_recurr_old_X*P_recurr_old_X')/den_recurr
#                 # update network matrix
#                 dW_recurr = -error(plas)*(P_recurr_old_X/den_recurr)'
#                 WXX(plas,pre_plastic_units(plas).inds) = WXX(plas,pre_plastic_units(plas).inds) + dW_recurr
#                 # store change in weights
#                 dW_recurr_len(i) = dW_recurr_len(i) + np.sqrt(dW_recurr*dW_recurr')

        if TRAIN_READOUT:
            pass
            # update inverse correlation matrix (using property P' = P)
#             P_readout_old = P_readout
#             P_readout_old_X = P_readout_old*X
#             den_readout = 1 + X'*P_readout_old_X
#             P_readout = P_readout_old - (P_readout_old_X*P_readout_old_X')/den_readout
#             # update error
#             error = Out - target_Out(i)
#             # update output weights
#             dW_readout = -error*(P_readout_old_X/den_readout)'
#             WXOut = WXOut + dW_readout
#             # store change in weights
#             dW_readout_len(i) = np.sqrt(dW_readout*dW_readout')

    # store output
    Out_history[:, i] = O
    X_history[:, [i]] = X
    WXOut_len[i] = np.sqrt(np.sum(np.square(WXOut[:])))
    WXX_len[i] = np.sqrt(np.sum(np.square(WXX[:])))

 27%|██▋       | 439/1600 [00:09<00:23, 49.17it/s]


KeyboardInterrupt: 

bookmark

In [6]:
importlib.reload(N)

<module 'network' from '/Users/michaelseay/Code/py-rrn/network.py'>

In [11]:
# setting parameters
GENERATOR_PARAMS = {'n_units': 800,
                 'p_plastic': 0.6,
                 'p_connect': 0.1,
                 'syn_strength': 1.5,}

TRIAL_PARAMS = {'length_ms': 1000,
                'spacing': 2,
                'time_step': 1,
                'start_train_ms': 250,
                'end_train_ms': 1400,}

INPUT_PARAMS = {'n_units': 1,
                'value': 5,
                'start_ms': 200,
                'duration_ms': 50}

OUTPUT_PARAMS = {'n_units': 1,
                 'value': 1,
                'center_ms': 1250,
                'width_ms': 30,
                'baseline_val': 0.2}

TRAIN_PARAMS = {'tau_ms': 10,
                'sigmoid': np.tanh,
                'noise_harvest': 0,
                'noise_train': 0.001,
                'n_trials_recurrent': 20,
                'n_trials_readout': 10,
                'n_trials_test': 10,
                }

# instantiating objects
GEN = N.Generator(**GENERATOR_PARAMS)
TRYAL = N.Trial(**TRIAL_PARAMS)
IN = N.Input(TRYAL, **INPUT_PARAMS)
OUT = N.Output(TRYAL, **OUTPUT_PARAMS)
TRAIN = N.Trainer(GEN, IN, OUT, TRYAL, **TRAIN_PARAMS)

In [12]:
TRAIN.initialize_weights()

In [13]:
TRAIN.harvest_innate()

In [10]:
%lprun -f TRAIN.harvest_innate TRAIN.harvest_innate()

In [111]:
%lprun -f main main()

100%|██████████| 1600/1600 [00:01<00:00, 1513.83it/s]


In [14]:
f, ax = plt.subplots()

imshow_cb(TRAIN.gen.innate, ax)

In [6]:
stage_params = {'innate': {'noise_amp': 0},
                'recurrent': {},
                'readout': {},
                'test': {},
               }

In [16]:
for i, in_vec in enumerate(IN.series.T):
    print(i, in_vec, IN.series[:, i])

0 [ 0.] [ 0.]
1 [ 0.] [ 0.]
2 [ 0.] [ 0.]
3 [ 0.] [ 0.]
4 [ 0.] [ 0.]
5 [ 0.] [ 0.]
6 [ 0.] [ 0.]
7 [ 0.] [ 0.]
8 [ 0.] [ 0.]
9 [ 0.] [ 0.]
10 [ 0.] [ 0.]
11 [ 0.] [ 0.]
12 [ 0.] [ 0.]
13 [ 0.] [ 0.]
14 [ 0.] [ 0.]
15 [ 0.] [ 0.]
16 [ 0.] [ 0.]
17 [ 0.] [ 0.]
18 [ 0.] [ 0.]
19 [ 0.] [ 0.]
20 [ 0.] [ 0.]
21 [ 0.] [ 0.]
22 [ 0.] [ 0.]
23 [ 0.] [ 0.]
24 [ 0.] [ 0.]
25 [ 0.] [ 0.]
26 [ 0.] [ 0.]
27 [ 0.] [ 0.]
28 [ 0.] [ 0.]
29 [ 0.] [ 0.]
30 [ 0.] [ 0.]
31 [ 0.] [ 0.]
32 [ 0.] [ 0.]
33 [ 0.] [ 0.]
34 [ 0.] [ 0.]
35 [ 0.] [ 0.]
36 [ 0.] [ 0.]
37 [ 0.] [ 0.]
38 [ 0.] [ 0.]
39 [ 0.] [ 0.]
40 [ 0.] [ 0.]
41 [ 0.] [ 0.]
42 [ 0.] [ 0.]
43 [ 0.] [ 0.]
44 [ 0.] [ 0.]
45 [ 0.] [ 0.]
46 [ 0.] [ 0.]
47 [ 0.] [ 0.]
48 [ 0.] [ 0.]
49 [ 0.] [ 0.]
50 [ 0.] [ 0.]
51 [ 0.] [ 0.]
52 [ 0.] [ 0.]
53 [ 0.] [ 0.]
54 [ 0.] [ 0.]
55 [ 0.] [ 0.]
56 [ 0.] [ 0.]
57 [ 0.] [ 0.]
58 [ 0.] [ 0.]
59 [ 0.] [ 0.]
60 [ 0.] [ 0.]
61 [ 0.] [ 0.]
62 [ 0.] [ 0.]
63 [ 0.] [ 0.]
64 [ 0.] [ 0.]
65 [ 0.] [ 0.]
66 [ 0.] [ 0.]
67 [ 

In [12]:
# creating the sparse random recurrent weight matrix WXX
# as well as (input => RRN) and (RRN => output) weights
GEN.initialize_weights()
IN.initialize_weights(GEN)
OUT.initialize_weights(GEN)

# assigning them to workspace names
WXX = GEN.wxx_ini
WInputX = IN.winputx_ini
WXOut = OUT.wxout_ini

# intializing some arrays which will store histories
# these are initalized OUTSIDE of the trial loop
X_history = np.zeros((GEN.n_units, TRYAL.n_steps))
Out_history = np.zeros((OUT.n_units, TRYAL.n_steps))

# initializing some array which will store training statistics
# these are initialized INSIDE of the trial loop
WXOut_len = np.zeros((TRYAL.n_steps))
WXX_len = np.zeros((TRYAL.n_steps))
dW_readout_len = np.zeros((TRYAL.n_steps))
dW_recurr_len = np.zeros((TRYAL.n_steps))

# setting for this particular run
TRAIN_RECURR = False
TRAIN_READOUT = False
use_noiseamp = 0
time_div = GEN.tau_ms / TRYAL.time_step

In [8]:
# creating initial conditions for firing rate, activation level, and output
Xv = 2 * np.random.rand(GEN.n_units, 1) - 1
X = GEN.sigmoid(Xv)
O = np.zeros((OUT.n_units,1))

# creating all noise ahead of time
all_noise = np.random.normal(scale=np.sqrt(TRYAL.time_step), size=(GEN.n_units, TRYAL.n_steps))

train_window = 0
for i in range(TRYAL.n_steps):

    in_vec = IN.series[:, i]
    noise = use_noiseamp * all_noise[:, [i]]
    Xv_current = WXX * X + WInputX * in_vec + noise
    Xv += (-Xv + Xv_current) / time_div
    X = GEN.sigmoid(Xv)
    O = np.dot(WXOut, X)
    
    # START SECTION FOR TRAINING IN RECURRENT / READOUT PARTS
    if (i == TRYAL.start_train_n):
        train_window = True
    if (i == TRYAL.end_train_n):
        train_window = False

    if train_window and i % TRYAL.spacing == 0:

        if TRAIN_RECURR:
            error = X - Target_innate_X[:, i]
            for plas in 1:GEN.n_plastic
                
        if TRAIN_READOUT:
            pass
    # END SECTION FOR TRAINING IN RECURRENT / READOUT PARTS

    Out_history[:, i] = O
    X_history[:, [i]] = X
    WXOut_len[i] = np.sqrt(np.sum(np.square(WXOut)))
    WXX_len[i] = np.sqrt(WXX.power(2).sum()) # note these are methods of sparse arrays
#     WXX_len[i] = np.sqrt(np.sum(np.square(WXX)))

100%|██████████| 1600/1600 [00:00<00:00, 1964.91it/s]


In [116]:
f = plt.figure()

ax1 = plt.subplot2grid((4, 1), (0, 0))
ax2 = plt.subplot2grid((4, 1), (1, 0), rowspan=2)
ax3 = plt.subplot2grid((4, 1), (3, 0))

# top panel
ax1.plot(TRYAL.time_ms, OUT.series.T, 'g')
ax1.plot(TRYAL.time_ms, IN.series.T / 2, 'b')
ax1.plot(TRYAL.time_ms, Out_history.T, 'r')

# middle panel
ten_trials = np.broadcast_to(np.expand_dims(TRYAL.time_ms, 1), (TRYAL.n_steps, 10))
ten_traces = X_history[:10, :].T + np.arange(10)*2
ax2.plot(ten_trials, ten_traces)

# bottom panel
ax3.plot(TRYAL.time_ms, WXOut_len)
# ax3.plot(TRYAL.time_ms, WXX_len)

In [110]:
# main loop
def main():
    # creating initial conditions for firing rate, activation level, and output
    Xv = 2 * np.random.rand(GEN.n_units, 1) - 1
    X = GEN.sigmoid(Xv)
    O = np.zeros((OUT.n_units,1))
    
    # creating all noise ahead of time
    all_noise = np.random.normal(scale=np.sqrt(TRYAL.time_step), size=(GEN.n_units, TRYAL.n_steps))
    
    train_window = 0
    for i in tqdm(range(TRYAL.n_steps)):

        in_vec = IN.series[:, i]
        noise = use_noiseamp * all_noise[:, [i]]
        Xv_current = WXX * X + WInputX * in_vec + noise
        Xv += (-Xv + Xv_current) / time_div
        X = GEN.sigmoid(Xv)
        O = np.dot(WXOut, X)

        if (i == TRYAL.start_train_n):
            train_window = True
        if (i == TRYAL.end_train_n):
            train_window = False

        if train_window and i % TRYAL.spacing == 0:

            if TRAIN_RECURR:
                error = X - Target_innate_X[:, i]
                for plas in 1:GEN.n_plastic

            if TRAIN_READOUT:
                pass

        Out_history[:, i] = O
        X_history[:, [i]] = X
        WXOut_len[i] = np.sqrt(np.sum(np.square(WXOut)))
        # next line is the most expensive line
        WXX_len[i] = np.sqrt(WXX.power(2).sum()) # convert to regular numpy array for speed

In [87]:
main()

100%|██████████| 1600/1600 [00:00<00:00, 5100.82it/s]


In [111]:
%lprun -f main main()

100%|██████████| 1600/1600 [00:01<00:00, 1513.83it/s]


In [ ]:
# raw matlab code

WXOut_len = np.zeros((1, TRYAL.n_steps))
WXX_len = np.zeros((1, TRYAL.n_steps))
dW_readout_len = np.zeros((1, TRYAL.n_steps))
dW_recurr_len = np.zeros((1, TRYAL.n_steps))
train_window = 0

# initial conditions
Xv = 1*(2*np.random.rand(numUnits,1)-1)
X = GEN.sigmoid(Xv)
Out = np.zeros(numOut,1)


# integration loop
for i = 1:n_steps

    if rem(i,round(n_steps/10)) == 0 && (TRAIN_RECURR == 1 || TRAIN_READOUT == 1)
        fprintf('.')
    end

    in_vec= input_pattern(:,i)

    # update units
    noise = use_noiseamp*np.random.normal(numUnits,1)*np.sqrt(TRYAL.time_step)
    Xv_current = WXX*X + WInputX*in_vec+ noise
    Xv = Xv + ((-Xv + Xv_current)./tau)*TRYAL.time_step
    X = GEN.sigmoid(Xv)
    Out = WXOut*X

    # start-end training window
    if (i == start_train_n)
        train_window = 1
    end
    if (i == end_train_n)
        train_window = 0
    end

    # training
    if (train_window == 1 && rem(i,learn_every) == 0)

        if TRAIN_RECURR == 1
            # train recurrent
            error = X - Target_innate_X(:,i)
            for plas = 1:numplastic_Units
                X_pre_plastic = X(pre_plastic_units(plas).inds)
                P_recurr_old = P_recurr(plas).P
                P_recurr_old_X = P_recurr_old*X_pre_plastic
                den_recurr = 1 + X_pre_plastic'*P_recurr_old_X
                P_recurr(plas).P = P_recurr_old - (P_recurr_old_X*P_recurr_old_X')/den_recurr
                # update network matrix
                dW_recurr = -error(plas)*(P_recurr_old_X/den_recurr)'
                WXX(plas,pre_plastic_units(plas).inds) = WXX(plas,pre_plastic_units(plas).inds) + dW_recurr
                # store change in weights
                dW_recurr_len(i) = dW_recurr_len(i) + np.sqrt(dW_recurr*dW_recurr')
            end
        end

        if TRAIN_READOUT == 1
            # update inverse correlation matrix (using property P' = P)
            P_readout_old = P_readout
            P_readout_old_X = P_readout_old*X
            den_readout = 1 + X'*P_readout_old_X
            P_readout = P_readout_old - (P_readout_old_X*P_readout_old_X')/den_readout
            # update error
            error = Out - target_Out(i)
            # update output weights
            dW_readout = -error*(P_readout_old_X/den_readout)'
            WXOut = WXOut + dW_readout
            # store change in weights
            dW_readout_len(i) = np.sqrt(dW_readout*dW_readout')
        end

    end
    # store output
    Out_history(:,i) = Out
    X_history(:,i) = X
    WXOut_len(i) = np.sqrt(sum(reshape(WXOut.^2,numOut*numUnits,1)))
    WXX_len(i) = np.sqrt(sum(reshape(WXX.^2,numUnits^2,1)))
end

# testing 1

In [4]:
# check: plot input and output patterns

print(TRYAL.time_ms.shape)
print(IN.series.shape)
print(Out.series.shape)

f, ax = plt.subplots()
ax.plot(TRYAL.time_ms, IN.series.T, label='input')
ax.plot(TRYAL.time_ms, Out.series.T, label='output')

ax.legend()

(1600,)
(1, 1600)
(1, 1600)


In [28]:
# quick verification that all of the following ways of creating
# a matrix containing random normal numbers
# are equivalent

prng = RandomState(1234) # note one needs to re-call this to get same results every time
test1 = prng.normal(size=(GEN.n_units, GEN.n_units)) * GEN.scale_recurr

prng = RandomState(1234)
test2 = prng.normal(scale=GEN.scale_recurr, size=(GEN.n_units, GEN.n_units))

prng = RandomState(1234)
test3 = norm.rvs(scale=GEN.scale_recurr, size=(GEN.n_units, GEN.n_units), random_state=prng)

In [ ]:
norm.pdf()

In [29]:
assert_allcombs_equal((test1, test2, test3), ck_fun=assert_array_equal)

In [ ]:
# testing equality / speed of generating random sparse matrices

# method 1

In [169]:
# %%timeit

# uniform distribution for mask
prng = RandomState(1234)
WXX_mask = prng.rand(GEN.n_units, GEN.n_units)
WXX_mask[WXX_mask <= GEN.p_connect] = 1
WXX_mask[WXX_mask < 1] = 0

# normal distribution for vals
prng = RandomState(1234)
WXX_vals = prng.normal(scale=GEN.scale_recurr, size=(GEN.n_units, GEN.n_units))

# create non-sparse version of WXX and set self-connections (diagonal elements) to 0
WXX_nonsparse = WXX_vals * WXX_mask
np.fill_diagonal(WXX_nonsparse, 0)
# WXX_nonsparse[np.diag_indices_from(WXX_nonsparse)] = 0

# convert to be sparse
# WXX = coo_matrix(WXX_nonsparse)
WXX = csr_matrix(WXX_nonsparse)

In [170]:
# method 2

In [171]:
rvs = norm(scale=GEN.scale_recurr).rvs

In [172]:
# %%timeit

# random variable generator object
prng = RandomState(1234)
WXX_s = random(GEN.n_units, GEN.n_units, density=GEN.p_connect, random_state=prng, data_rvs=rvs,
               format='csr',)
#                format='csr',)
# WXX_s[np.diag_indices_from(WXX_s)] = 0
WXX_s.setdiag(0)

/Users/michaelseay/anaconda/lib/python3.5/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [173]:
cm = WXX_nonsparse
print(cm.shape, np.count_nonzero(cm), cm.min(), cm.max())

(800, 800) 63797 -0.775266142687 0.726642624784


In [174]:
ck_mats = (WXX, WXX_s)

for cm in ck_mats:
    print(cm.shape, cm.nnz, cm.count_nonzero(), cm.min(), cm.max())

(800, 800) 63797 63797 -0.775266142687 0.726642624784
(800, 800) 64723 63923 -0.742525396081 0.73124373277


In [96]:
f, ax = plt.subplots()
imshow_cb(WXX.toarray(), ax)

In [95]:
f, ax = plt.subplots()
imshow_cb(WXX_s.toarray(), ax)

In [ ]:
# second method is slower :(

# testing 2

In [ ]:
ck_mats = (WXX_nonsparse, WXX, WXX_c, WXX_o, WInputX, WXOut)

In [ ]:
# testing speed of *

In [109]:
%%timeit
test1_ns = WXX_nonsparse*X # SLOW, WRONG SHAPE

1000 loops, best of 3: 700 µs per loop


In [110]:
%%timeit
test1 = WXX*X

The slowest run took 4.41 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 60.6 µs per loop


In [111]:
%%timeit
test1_c = WXX_c*X # NO DIFFERENCE WITH ROW / COLUMN SPARSE MATRICES

10000 loops, best of 3: 62.5 µs per loop


In [112]:
%%timeit
test1_o = WXX_o*X # COO sparse matrix is slower

10000 loops, best of 3: 159 µs per loop


In [133]:
test1_ns = WXX_nonsparse*X # SLOW, WRONG SHAPE
test1 = WXX*X
test1_c = WXX_c*X # NO DIFFERENCE WITH ROW / COLUMN SPARSE MATRICES
test1_o = WXX_o*X # COO sparse matrix is slower

print(test1_ns.shape)
print(test1.shape)
print(test1_c.shape)
print(test1_o.shape)

(800, 800)
(800, 1)
(800, 1)
(800, 1)


In [116]:
ck_mats = (test1, test1_c, test1_o)

In [118]:
assert_allcombs_equal(ck_mats, assert_array_equal)

In [119]:
# testing speed of dot

In [126]:
%%timeit
test2_ns = WXX_nonsparse.dot(X) # ONLY VERY SLIGHTLY SLOWER THAN
# SPARSE, CORRECT

The slowest run took 5.99 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 69.7 µs per loop


In [127]:
%%timeit
test2 = WXX.dot(X)

10000 loops, best of 3: 63.1 µs per loop


In [129]:
%%timeit
test2_c = WXX_c.dot(X)

The slowest run took 4.52 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 62.4 µs per loop


In [128]:
%%timeit
test2_o = WXX_o.dot(X)

10000 loops, best of 3: 160 µs per loop


In [134]:
test2_ns = WXX_nonsparse.dot(X) # ONLY VERY SLIGHTLY SLOWER THAN
# SPARSE, CORRECT BUT NUMERICALLY SLIGHTLY DIFFERENT?
test2 = WXX.dot(X)
test2_c = WXX_c.dot(X)
test2_o = WXX_o.dot(X)

In [135]:
ck_mats = (test1, test1_c, test1_o, test2, test2_c, test2_o, test2_ns)

In [140]:
ck_mats = (test2, test2_c, test2_o,)

In [136]:
assert_allcombs_equal(ck_mats)

In [141]:
assert_allcombs_equal(ck_mats, ck_fun=assert_array_equal)

In [ ]:
# testing speed of matmul

In [145]:
%%timeit
test3_ns = np.matmul(WXX_nonsparse, X)

The slowest run took 7.55 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 69.5 µs per loop


In [148]:
%%timeit
test3 = np.matmul(WXX, X)

TypeError: Object arrays are not currently supported

In [147]:
test3_ns.shape

(800, 1)

In [ ]:
# conclusion, for multiplying the sparse matrix by a vector, it's fastest
# for a csr_matrix and just use the * operator, it will do the matrix
# multiplication in about 60 us

# testing 3

fastest way to square, sum, and square root a sparse matrix

In [95]:
%%timeit
np.sqrt(np.sum(np.square(WXX)))

10 loops, best of 3: 26.6 ms per loop


In [96]:
%%timeit
np.sqrt(np.sum(np.square(WXX.toarray())))

100 loops, best of 3: 2 ms per loop


In [109]:
%%timeit
np.sqrt(WXX.power(2).sum())

The slowest run took 5.97 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 263 µs per loop


In [105]:
t1 = np.sqrt(np.sum(np.square(WXX)))
t2 = np.sqrt(np.sum(np.square(WXX.toarray())))
t3 = np.sqrt(WXX.power(2).sum())

In [115]:
# conclusion: use method 3